In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import cv2

import functions as fn

In [2]:
dataDir='../data'
trainDir='train2017'
valDir='val2017'
seed=42
transformed=.5
train=.6
val=.2
test = 1 - train - val

np.random.seed(seed)

trainAnn=f'{dataDir}/annotations/instances_{trainDir}.json'
valAnn=f'{dataDir}/annotations/instances_{valDir}.json'

cocoTrain=COCO(trainAnn)
cocoVal=COCO(valAnn)

catIdsTrain = cocoTrain.getCatIds(catNms=['person'])
imgIdsTrain = cocoTrain.getImgIds(catIds=catIdsTrain)
imgIdsTrain = cocoTrain.getImgIds(imgIds=imgIdsTrain)
annIdsTrain = cocoTrain.getAnnIds(imgIds=imgIdsTrain, catIds=catIdsTrain, iscrowd=None)

catIdsVal = cocoVal.getCatIds(catNms=['person'])
imgIdsVal = cocoVal.getImgIds(catIds=catIdsVal)
imgIdsVal = cocoVal.getImgIds(imgIds=imgIdsVal)
annIdsVal = cocoVal.getAnnIds(imgIds=imgIdsVal, catIds=catIdsVal, iscrowd=None)

loading annotations into memory...
Done (t=9.79s)
creating index...
index created!
loading annotations into memory...
Done (t=0.34s)
creating index...
index created!


In [3]:
catIdsTrain == catIdsVal

True

In [4]:
#foo[mdpt:blacks[-1]] = right_arr

In [5]:
len(imgIdsTrain), len(imgIdsVal)

(64115, 2693)

In [6]:
#imgIds = np.append(imgIdsTrain, imgIdsVal)
#len(imgIds)

In [7]:
train, val, test

(0.6, 0.2, 0.2)

In [8]:
#annIds = cocoTrain.getAnnIds(imgIds=imgIdsTrain, catIds=catIdsTrain, iscrowd=None)

In [9]:
#len(annIds)

In [29]:
count = 0
limit = 20

idx = 69

testImg = cocoTrain.loadImgs(imgIdsTrain[idx])[0]
testAnn = cocoTrain.loadAnns(cocoTrain.getAnnIds(imgIds=testImg['id'], catIds=catIdsTrain, iscrowd=None))

fpath = '{}/{}/{}'.format(dataDir, trainDir, testImg['file_name'])

testImgArr = cv2.imread(fpath)

fn.remove_true_blacks(testImgArr)

testSpecAnn = testAnn[1] # np.random.randint(0, len(testAnn))

maskedTest, maskedTestInv = fn.generate_masked_images(testImgArr, testSpecAnn)

#testSpecAnn = fn.coco_polyseg_formatted(testSpecAnn)

#newMaskedTest = fn.apply_rolling_shutter(maskedTest, testSpecAnn, strategy='left')

#newMaskedTestInv = fn.fill_inv_masked(maskedTestInv, testSpecAnn)

fn.display_image(testImgArr)

In [43]:
def get_ann_dims(ann):
    return int(np.round(ann['bbox'][0])), int(np.round(ann['bbox'][1])), int(np.round(ann['bbox'][2])), int(np.round(ann['bbox'][3])) # xmin, ymin, width, height


def fill_inv_masked(inv_masked_img, ann, strategy='right'): # when strategy='right', pull from left; when strategy='left', pull from right
    output = np.copy(inv_masked_img)

    xmin, ymin, width, height = get_ann_dims(ann)
    id = ann['id']
    atLeft, atRight = False, False

    if strategy not in ['left', 'right']:
        raise ValueError(f'Strategy {strategy} is invalid')

    if xmin == 0: # at left side of image
        atLeft = True
    if xmin + width == inv_masked_img.shape[1]: # at right side of image, pull from left
        atRight = True
    
    if atLeft:
        if atRight:
            raise ValueError(f'Image #{id} takes up entire image width, cannot fill')
        else:
            strategy='left'
    elif atRight:
        strategy='right'

    if strategy == 'right':
        print(strategy)
        for row in np.arange(ymin, ymin + height):
            blacks = np.sort(np.array(list(set(np.where(output[row] == np.array([0, 0, 0]))[0]))))
            if(len(blacks) > 0):
                output[row][blacks[0]:blacks[-1] + 1] = np.flip(output[row][blacks[0] - len(blacks):blacks[0]], axis=0)
    else:
        for row in np.arange(ymin, ymin + height):
            blacks = np.sort(np.array(list(set(np.where(output[row] == np.array([0, 0, 0]))[0]))))
            if(len(blacks) > 0):
                output[row][blacks[0]:blacks[-1] + 1] = np.flip(output[row][blacks[-1] + 1:blacks[-1] + 1 + len(blacks)], axis=0)

    #else:
    #    print('from center')


    #for row in np.arange(ymin, ymax):
    #    blacks = np.sort(np.array(list(set(np.where(output[row] == np.array([0, 0, 0]))[0]))))
    #    mdpt = blacks[int(len(blacks) / 2)]
    #    left_len = mdpt - blacks[0]
    #    right_len = blacks[-1] - mdpt
    #    left_arr = np.flip(output[row][blacks[0] - left_len:blacks[0]], axis=0)
    #    right_arr = np.flip(output[row][blacks[-1] + 1:blacks[-1] + right_len + 2], axis=0)
    #    output[row][blacks[0]:mdpt] = left_arr
    #    output[row][mdpt:blacks[-1] + 1] = right_arr
    
    return output

grand = fill_inv_masked(maskedTestInv, testSpecAnn)

In [44]:
fn.display_image(grand)

In [26]:
testSpecAnn['id']

502933

In [17]:
maskedTest.shape[1]

500

In [18]:
fn.display_image(maskedTest)

In [19]:
# 3, 48 is good center
# 4, 69 is good left
# 21.1 is good wide lol
# 60.1 is good right 

zidx = 60

ztestImg = cocoTrain.loadImgs(imgIdsTrain[zidx])[0]
ztestAnn = cocoTrain.loadAnns(cocoTrain.getAnnIds(imgIds=ztestImg['id'], catIds=catIdsTrain, iscrowd=None))

zfpath = '{}/{}/{}'.format(dataDir, trainDir, ztestImg['file_name'])

ztestImgArr = cv2.imread(zfpath)

fn.remove_true_blacks(ztestImgArr)

fn.display_image(ztestImgArr)


In [20]:
ztestSpecAnn = ztestAnn[1]

zmaskedTest, zmaskedTestInv = fn.generate_masked_images(ztestImgArr, ztestSpecAnn)

fn.display_image(zmaskedTest)

In [64]:
zmaskedTest.shape

(640, 416, 3)

In [77]:
#fn.coco_polyseg_formatted(ztestSpecAnn)

In [39]:
zrand = fill_inv_masked(zmaskedTestInv, ztestSpecAnn)

right


In [40]:
fn.display_image(zrand)

In [128]:
znewMaskedTestInv = fn.fill_inv_masked(zmaskedTestInv, ztestSpecAnn) # 1

znewMaskedTest = fn.apply_rolling_shutter(zmaskedTest, ztestSpecAnn, interval=2, strategy='right')

zoutput = fn.recombine_masked_imgs(znewMaskedTest, znewMaskedTestInv) # 1

#zrecombined = fn.recombine_masked_imgs(znewMaskedTest, zmaskedTestInv) # 2

#zoutput = fn.fill_inv_masked(zrecombined, ztestSpecAnn) # 2

rand, rand2 = fn.generate_masked_images(zoutput, ztestSpecAnn)

fn.display_image(zoutput)

In [48]:
testImg

{'license': 1,
 'file_name': '000000000136.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000000136.jpg',
 'height': 374,
 'width': 500,
 'date_captured': '2013-11-17 01:42:35',
 'flickr_url': 'http://farm1.staticflickr.com/90/206826531_339a965783_z.jpg',
 'id': 136}

In [17]:
testAnn = cocoTrain.loadAnns(cocoTrain.getAnnIds(imgIds=testImg['id'], catIds=catIdsTrain, iscrowd=None))

In [20]:
len(testAnn)

2

In [41]:
np.random.seed(seed)

for i in range(10):
    print(np.random.randint(0, len(testAnn)))

0
1
0
0
0
1
0
0
0
1
